In [1]:
#funcao que resolve o problema da soma do subconjunto super crescente
def supIncSum(seq, s):
    n = len(seq) - 1
    res = []
    while n >= 0:
        if s >= seq[n]:
            res.append(1)
            s -= seq[n]
        else:
            res.append(0)
        n -= 1
    res.reverse()
    return res

In [2]:
# constroi uma sequencia super crescente dado o tamanho da seq
def randomSupIncSequence(n):
    soma = 0
    first = randint(1, 2 ** (n+1))
    f = lambda x : x * randint(2, n) + randint(1, 2 ** n)
    size = 1
    res = [first]
    soma += first
    for i in range(n-1):
        first = f(first)
        res.append(first)
        soma += first
    return res, soma

In [3]:
#Encontra M dado o valor total da soma de seq
def findM(soma, n):
    return randint(1, n) * soma + randint(1, 2 ** n)



#Encontra um numero coprimo com o parametro (algoritmo brute force)
def findW(M):
    W = randrange(1, M)
    while gcd(M, W) != 1:
        W += 1
    return W

In [4]:
#Gerador de Permutações aleatorias
def sattoloShuffle(n):
    res = list(range(n))
    while n > 1:
        n -= 1
        j = randrange(n)
        res[j], res[n] = res[n], res[j]
    return res

In [5]:
#Geracao de chave simples
def KSKeyGen(n):
    seq, soma = randomSupIncSequence(n)
    M = findM(soma, n)
    zM = IntegerModRing(M)
    W = zM(findW(M))
    perm = sattoloShuffle(n)
    privKey = (n, perm, M, W, seq)
    pubKeylist = []
    for i in range(n):
        pubKeylist.append(ZZ(W * seq[perm[i]]))
    pubKey = (n, pubKeylist)
    print("Limite de cifracao: " + str(2 ** n - 1))
    return pubKey, privKey

In [6]:
p, P = KSKeyGen(32)

Limite de cifracao: 4294967295


In [7]:
def cifravel(mens, n):
    return mens <= 2 ** n - 1

#Encriptacao Simples
def KSEncrypt(mensagem, pubKey):
    n, pubKeylist = pubKey
    if cifravel(mensagem, n):
        c = 0
        for ind, i in enumerate(map(int, format(mensagem, "0" + str(n) + "b"))):
            c += i * pubKeylist[ind]
        return c
    else:
        print("A mensagem ultrapassa o limite de cifracao")
        return []

In [8]:
#Decriptacao Simples
def KSDecrypt(criptograma, privKey):
    n, perms, M, W, seq = privKey
    zM = IntegerModRing(M)
    wMod = zM(W)
    d = wMod ^ -1 * criptograma
    bits = supIncSum(seq, d)
    msg = 0
    for i in range(n):
        bit = bits[perms[i]]
        msg += bit * 2 ** (n-1)
        n -= 1
    return msg

In [9]:
for k in range(1,30000):
    if k != KSDecrypt(KSEncrypt(k, p), P):
        print(k)

In [10]:
#Geracao de chave Multi Iterada
def MIKSKeyGen(n, t):
    seq, soma = randomSupIncSequence(n)
    seqj = seq
    Ms = []
    Ws = []
    for j in range(t):
        somaj = sum([ZZ(a) for a in seqj])
        Mj = findM(somaj, n)
        Ms.append(Mj)
        Wj = findW(Mj)
        Ws.append(Wj)
        zMj = IntegerModRing(Mj)
        seqj = [a * Wj for a in map(zMj, seqj)]
    perms = sattoloShuffle(n)
    pubKeylist = []
    for i in range(n):
        pubKeylist.append(seqj[perms[i]])
    pubKey = n, pubKeylist
    privKey = n, perms, Ms, Ws, seq
    print("Limite de cifracao: " + str(2 ** n - 1))
    return pubKey, privKey

In [11]:
p, P = MIKSKeyGen(1024, 126) # segundo argumento faz tempo de execucao disparar

Limite de cifracao: 179769313486231590772930519078902473361797697894230657273430081157732675805500963132708477322407536021120113879871393357658789768814416622492847430639474124377767893424865485276302219601246094119453082952085005768838150682342462881473913110540827237163350510684586298239947245938479716304835356329624224137215


In [12]:
#Encriptacao Multi Iterada
def MIKSEncrypt(mensagem, pubKey):
    return KSEncrypt(mensagem, pubKey)

In [13]:
#Decriptacao Multi Iterada
def MIKSDecrypt(cripto, privKey):
    n, perms, Ms, Ws, seq = privKey
    dj = cripto
    for Mj, Wj in zip(Ms[::-1], Ws[::-1]):
        zMj = IntegerModRing(Mj)
        Wjmod = zMj(Wj)
        djmod = zMj(dj)
        dj = Wjmod ** -1 * djmod
    bits = supIncSum(seq, dj)
    msg = 0
    for i in range(n):
        bit = bits[perms[i]]
        msg += bit * 2 ** (n-1)
        n -= 1
    return msg

In [14]:
MIKSDecrypt(MIKSEncrypt(332123, p), P)

332123